In [1]:
from example import f_m_func

f_m = f_m_func()
print(f_m)

#f_m is a function for a specific measurement, which connects *_sp sensor pose with land mark observaion *_lm

-log(sigma_d) - log(sigma_fa) - log(x_lm**2 - 2*x_lm*x_sp + x_sp**2 + y_lm**2 - 2*y_lm*y_sp + y_sp**2)/2 - log(pi) - 0.693147180559946 - 0.5*(sigma_d**2*(mu_fa - atan2((-x_lm + x_sp)*sin(yaw_sp) + (y_lm - y_sp)*cos(yaw_sp), (2*(x_lm - x_sp)*cos(yaw_sp)**2 + (y_lm - y_sp)*sin(2*yaw_sp))/(2*cos(yaw_sp))))**2 + sigma_fa**2*(mu_d - sqrt(x_lm**2 - 2*x_lm*x_sp + x_sp**2 + y_lm**2 - 2*y_lm*y_sp + y_sp**2))**2)/(sigma_d**2*sigma_fa**2)


In [2]:
from example import eq_factory, unify_eqs

#we have observation from 2 distinct sensor poses
sp0, sp1 = eq_factory(f_m, ['x_sp', 'y_sp', 'yaw_sp'], idx=range(2))

eqs = [sp0, sp1]

eqs = unify_eqs(eqs, ['mu_fa', 'mu_d'])

for eq in eqs:
    print (eq)
    print ('----')

-log(sigma_d) - log(sigma_fa) - log(x_lm**2 - 2*x_lm*x_sp_0 + x_sp_0**2 + y_lm**2 - 2*y_lm*y_sp_0 + y_sp_0**2)/2 - log(pi) - 0.693147180559946 - 0.5*(sigma_d**2*(mu_fa_0 - atan2((-x_lm + x_sp_0)*sin(yaw_sp_0) + (y_lm - y_sp_0)*cos(yaw_sp_0), (2*(x_lm - x_sp_0)*cos(yaw_sp_0)**2 + (y_lm - y_sp_0)*sin(2*yaw_sp_0))/(2*cos(yaw_sp_0))))**2 + sigma_fa**2*(mu_d_0 - sqrt(x_lm**2 - 2*x_lm*x_sp_0 + x_sp_0**2 + y_lm**2 - 2*y_lm*y_sp_0 + y_sp_0**2))**2)/(sigma_d**2*sigma_fa**2)
----
-log(sigma_d) - log(sigma_fa) - log(x_lm**2 - 2*x_lm*x_sp_1 + x_sp_1**2 + y_lm**2 - 2*y_lm*y_sp_1 + y_sp_1**2)/2 - log(pi) - 0.693147180559946 - 0.5*(sigma_d**2*(mu_fa_1 - atan2((-x_lm + x_sp_1)*sin(yaw_sp_1) + (y_lm - y_sp_1)*cos(yaw_sp_1), (2*(x_lm - x_sp_1)*cos(yaw_sp_1)**2 + (y_lm - y_sp_1)*sin(2*yaw_sp_1))/(2*cos(yaw_sp_1))))**2 + sigma_fa**2*(mu_d_1 - sqrt(x_lm**2 - 2*x_lm*x_sp_1 + x_sp_1**2 + y_lm**2 - 2*y_lm*y_sp_1 + y_sp_1**2))**2)/(sigma_d**2*sigma_fa**2)
----


In [4]:
import numpy as np
import os
parentdir = os.path.dirname(os.path.dirname(os.path.abspath('')))
parentdir = os.path.dirname(os.path.dirname(parentdir))
os.sys.path.insert(0, parentdir) 
from equations.methodology.pytorchsymbolic.sympytorch import SymPyModule
import torch

import matplotlib.pyplot as plt

init_vals = {
    'x_sp_0': 0,
    'y_sp_0': 0,
    'yaw_sp_0': 0,

    'x_sp_1': 0,
    'y_sp_1': 0,
    'yaw_sp_1': 0,

    'x_lm': 1.,
    'y_lm': 1.,
}

sm = SymPyModule(eqs, init_vals=init_vals)

x, y = np.mgrid[0:6:0.01, -2:2:0.01]

optimizer = torch.optim.SGD(sm.parameters(), lr=1e-3)

for i in range(100):
    optimizer.zero_grad()

    log_p = sm(mu_fa_0=torch.Tensor([0.]),   mu_d_0=torch.Tensor([5.]), 
               mu_fa_1=torch.Tensor([np.pi]), mu_d_1=torch.Tensor([5.]),
               sigma_fa=torch.Tensor([0.1]), sigma_d=torch.Tensor([0.1]))

    loss = -log_p.mean()

    print (loss)

    loss.backward()
    optimizer.step()
    
for name, param in sm.named_parameters():
    print (name, param)



_nodes.0._args.4._args.1._args.0._args.0._args.0.x_lm Parameter containing:
tensor(1., requires_grad=True)
_nodes.0._args.4._args.1._args.0._args.1._args.0.x_sp_0 Parameter containing:
tensor(0., requires_grad=True)
_nodes.0._args.4._args.1._args.0._args.2._args.0.y_lm Parameter containing:
tensor(1., requires_grad=True)
_nodes.0._args.4._args.1._args.0._args.3._args.0.y_sp_0 Parameter containing:
tensor(0., requires_grad=True)
_nodes.0._args.5._args.3._args.0._args.1._args.0._args.1._args.1._args.0._args.0._args.1._args.0.yaw_sp_0 Parameter containing:
tensor(0., requires_grad=True)
_nodes.1._args.4._args.1._args.0._args.1._args.0.x_sp_1 Parameter containing:
tensor(0., requires_grad=True)
_nodes.1._args.4._args.1._args.0._args.3._args.0.y_sp_1 Parameter containing:
tensor(0., requires_grad=True)
_nodes.1._args.5._args.3._args.0._args.1._args.0._args.1._args.1._args.0._args.0._args.1._args.0.yaw_sp_1 Parameter containing:
tensor(0., requires_grad=True)
tensor(794.6851, grad_fn=<NegBac